<a href="https://colab.research.google.com/github/patsoong/CS506FinalProject/blob/main/Logistic_regressor_baseline_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import pandas as pd

features_df = pd.read_csv() #needs implementing

num_cols = features_df.select_dtypes(include="number").columns.tolist() #remove features not to be used in training data
for col in ["won_title", "season"]:
    if col in num_cols:
        num_cols.remove(col)

X = features_df[num_cols].copy()
y = features_df["won_title"].astype(int)

train_mask = features_df["season"] <= 2015
X_train, X_test = X[train_mask], X[~train_mask]
y_train, y_test = y[train_mask], y[~train_mask]

id_test = features_df.loc[~train_mask, ["season", "team", "won_title"]].copy()

model = Pipeline([
    ("scale", StandardScaler(with_mean=True, with_std=True)),
    ("clf", LogisticRegression(multi_class='ovr', class_weight='balanced', max_iter=10000))
])

model.fit(X_train, y_train)
probabilities = model.predict_proba(X_test)[:, 1]

id_test["proba_win"] = probabilities

true_champs = id_test[id_test["won_title"] == 1][["season", "team"]]
predicted_champs = id_test.groupby("season").apply(lambda df: df.nlargest(1, "proba_win"))
eval = predicted_champs.merge(true_champs, on="season", suffixes=("_pred", "_true"))
eval["correct"] = (eval["team_pred"] == eval["team_true"]).astype(int)
print("Top-1 accuracy:", eval["correct"].mean())